In [1]:
import animate
import numpy as np
import pandas as pd
project_dir = '../data/'

In [2]:
games = pd.read_csv(f'{project_dir}games.csv')
play_df = pd.read_csv(f'{project_dir}plays.csv')
player_plays = pd.read_csv(f'{project_dir}player_play.csv')
players = pd.read_csv(f'{project_dir}players.csv')
tracking_df = pd.read_csv(f'{project_dir}tracking_week_1.csv')

# Notes

- Need to calculate number of players in box at snap (and dummy for each player yes/no)
- Find avg speed, speed at snap, distance from line of scrimmage, distance from qb at snap
- Find distance from initial starting position and at snap position for every defender
- Yard number, score, time, yards to go per play
- orientation at snap
- number of linemen on play
- number of LBs
- number of corners/safeties

In [4]:
player_df = pd.merge(player_plays, players, on='nflId')

In [8]:
# Defensive positions
def_positions = ['DT', 'LB', 'ILB', 'MLB', 'OLB', 'CB', 'SS', 'FS', 'DE', 'NT', 'DB']

def_positions_w_qb = def_positions + ['QB']

# filter by defensive positions
def_players = player_df[player_df['position'].isin(def_positions_w_qb)]

tracking_df = pd.merge(tracking_df, def_players[['gameId', 'playId', 'nflId']], 
    on=['gameId', 'playId', 'nflId'], how='inner')

player_plays = pd.merge(player_plays, def_players[['gameId', 'playId', 'nflId']], 
    on=['gameId', 'playId', 'nflId'], how='inner')

In [12]:
# Group by gameId, playId and count number of linemen, LB, CBs, and Safeties
def_players.groupby(['gameId', 'playId']).agg(
    num_linemen = ('position', lambda x: sum(x.isin(['DE', 'DT']))),
    num_LB = ('position', lambda x: sum(x.isin(['LB', 'ILB', 'MLB', 'OLB']))),
    num_CB = ('position', lambda x: sum(x.isin(['CB']))),
    num_S = ('position', lambda x: sum(x.isin(['SS', 'FS']))),
    num_DB = ('position', lambda x: sum(x.isin(['DB']))),
    num_players = ('position', 'count')
)

num_linemen  num_LB  num_CB  num_S  num_DB  num_players
gameId     playId                                                         
2022090800 56                3       2       2      3       0           12
           80                3       2       2      3       0           12
           101               3       2       3      2       0           12
           122               3       2       3      2       0           12
           167               3       2       3      2       0           12
...                        ...     ...     ...    ...     ...          ...
2022110700 3658              5       1       3      2       0           12
           3686              4       3       2      2       0           12
           3707              5       3       1      2       0           12
           3740              5       3       1      2       0           12
           3787              2       4       2      3       0           12

[16124 rows x 6 columns]